In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd



os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
metadata_train_df = pd.read_csv(r'C:\Users\nilss\Documents\GitHub\Melanoma\ernst\merged_model\real_train.csv')
metadata_val_df = pd.read_csv(r'C:\Users\nilss\Documents\GitHub\Melanoma\ernst\merged_model\real_val.csv')


metadata_df = pd.concat([metadata_train_df,metadata_val_df])

metadata_df.fillna(0.49) #Teils wurd kein Alter angegeben, dies Lücken werden mit dem normalisierten Durchschnittsalter gefüllt


,female,male,as1,as2,as3,as4,as5,as6,as7,as8,img_name,age,target
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_2637011,0.45,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_0015719,0.45,0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,ISIC_0052212,0.50,0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,ISIC_0068279,0.45,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,ISIC_0074268,0.55,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6643,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,ISIC_9999134,0.50,0
6644,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,ISIC_9999320,0.65,0
6645,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,ISIC_9999515,0.20,0
6646,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,ISIC_9999666,0.50,0


Bilder und Metadaten in einem NPZ File zusammenführen und speichern

Muss natürlich nicht jedes mal aufs neue gemacht werden.


In [ ]:
import math
import cv2
img_dir_path = r"C:\Users\nilss\Sets\images_v4_trainaug/" #Hier sind sämtliche Bilder in einem Ordner



for i, row in metadata_df.iterrows():
    img_path = img_dir_path+row['img_name']+'.jpg'


    npz_path = 'npzs/'+row['img_name']+'.npz'
   
    
    pic_bgr_arr = cv2.imread(img_dir_path+row['img_name']+'.jpg')
    pic_rgb_arr = cv2.cvtColor(pic_bgr_arr, cv2.COLOR_BGR2RGB)
    
    male,female,age = row['male'], row['female'],row['age'] #Zuteilen der Features von der CSV zum NPZ File.
    as1,as2,as3,as4,as5,as6,as7,as8= row['as1'], row['as2'], row['as3'], row['as4'], row['as5'], row['as6'], row['as7'], row['as8']

    if math.isnan(age):
        age=0.49 #Teils wurd kein Alter angegeben, dies Lücken werden mit dem normalisierten Durchschnittsalter gefüllt
        print(row['img_name']) #Zur Überwachung
    metadata = np.array([male,female,age,as1,as2,as3,as4,as5,as6,as7,as8])

    target = row['target']
    #np.savez_compressed(npz_path, img=pic_rgb_arr, metadata=metadata, target=target) #Zur sicherheit auskommentiert!
    

In [2]:
metadata_df.drop(
   ['as1','as2','as3','as4','as5','as6','as7','as8','as8','female','male','age'], #Diese Features brauchen wir nicht mehr.
axis=1,
inplace=True
    )
metadata_df.head()  #Kontrolle

,img_name,target
0,ISIC_2637011,0
1,ISIC_0015719,0
2,ISIC_0052212,0
3,ISIC_0068279,0
4,ISIC_0074268,0


In [8]:
metadata_df.shape

(33596, 2)

In [3]:


train_df,val_df = metadata_df[:26948],metadata_df[26948:] #Teilen in Val -und Trainset.


In [4]:
train_df.shape,val_df.shape

((26948, 2), (6648, 2))

In [ ]:
#Die Verteilung an Positiven Einträgen konrollieren.

train_malignant=0
train_benign=0
for i,row in train_df.iterrows():
    if row['target']==1:
        train_malignant+=1
    elif  row['target']==0:
        train_benign+=1
    else:
        print(row['target'])   

val_malignant=0
val_benign=0

for i,row in val_df.iterrows():
    if row['target']==1:
        val_malignant+=1
    else:
        val_benign+=1

'Train:',train_malignant,'malignants',train_benign,'benigns','Val:',val_malignant,'malignants',val_benign,'benigns'

In [37]:
#Funktion zum Einlesen der NPZ Dateien
def get_X_y(df,path,):
    """Fügt Metadaten und Bilddaten in einer NPZ-Datein zusammen. Gibt X_img, X_metadata und y aus.
    Params:\n \n
    - df, das DF dessen Eitträge vorgeben, welche NPZs Eingelesen werden sollen.\n \n
    - path, von wo die NPZs geladen werden sollen.Slash am Ende nicht vergessen!
    """
    assert path[-1]=='/'

    if type(df)==list:
        img_names = df
    else:
        print(type(df))
        img_names = df['img_name']    
    X_img,X_metadata = [],[]
    y=[]

    
    for name in img_names:
       
        npz= np.load(path+name.split('.')[0]+'.npz') 

        img = npz['img']
        X_img.append(img)

        metadata = npz['metadata']
        X_metadata.append(metadata)

        y.append(npz['target'])
        

    X_img,X_metadata = np.array(X_img),np.array(X_metadata)
    y=np.array(y)

    return (X_img,X_metadata),  y



In [ ]:
#Erstellen der Trainingssets und Kontrolle der Shapes
(X_train_img,X_train_metadata), y_train = get_X_y(train_df)


X_train_img.shape,X_train_metadata.shape,y_train.shape


In [7]:
#Erstellen der Trainingssets und Kontrolle der Shapes
(X_val_img,X_val_metadata), y_val = get_X_y(val_df)


X_val_img.shape,X_val_metadata.shape,y_val.shape

((6648, 224, 224, 3), (6648, 11), (6648,))

In [6]:
import tensorflow as tf
from tensorflow.keras import layers

CNN für Bilder


In [7]:

input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB3(include_top=False) #B3 wird hierverwendet.
base_model.trainable = False

inputs =layers.Input(shape=input_shape, name="input_layer")

x= base_model(inputs)

x= layers.GlobalAveragePooling2D()(x)

outputs = tf.keras.layers.Dense(15, #15 Neuronen als Output des Imagestreams.
activation="LeakyReLU", #LeakyReLU um "DyingReLU" zu umgehen.
 name="output_layer",
 
)(x)

cnn = tf.keras.Model(inputs, outputs) #Imagestream-Modell 


In [3]:
cnn.layers[-1].weights #Zur kontrolle, dass Gewichte frisch initialisiert wurden.

[<tf.Variable 'output_layer/kernel:0' shape=(1536, 15) dtype=float32, numpy=
 array([[-0.00722729,  0.006885  , -0.00850381, ...,  0.0603005 ,
          0.0077204 ,  0.050406  ],
        [ 0.03073881,  0.00521204, -0.00171753, ...,  0.06099519,
         -0.00855669,  0.05542484],
        [-0.05522988,  0.05413732,  0.04397985, ...,  0.01915576,
         -0.01350201,  0.02967417],
        ...,
        [ 0.02535794, -0.01607182,  0.04667811, ..., -0.01069925,
          0.03947179,  0.0588019 ],
        [ 0.03371282, -0.05033345, -0.02123518, ..., -0.04651993,
         -0.0011714 ,  0.05995372],
        [-0.01633942,  0.05476103, -0.01844093, ..., -0.01948228,
         -0.04969653, -0.00126596]], dtype=float32)>,
 <tf.Variable 'output_layer/bias:0' shape=(15,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)>]

FNN für Metadata

In [8]:
def stat_scaler(tensor):
   return tensor
input_stats = layers.Input(shape=(11,))

y = layers.Dense(72, activation="LeakyReLU",

#kernel_regularizer=tf.keras.regularizers.L1L2(l1=0.0005, l2=0.0005)
)(input_stats)
y = layers.Dense(5, 
activation="LeakyReLU",

#kernel_regularizer=tf.keras.regularizers.L1L2(l1=0.0005, l2=0.0005)
)(y)
fnn = tf.keras.Model(inputs=input_stats, outputs=y)

Zusammenführen der beiden (unter-) Modellen


In [9]:
merging_layer = layers.concatenate([cnn.output,fnn.output])#hierwerden die modelle zuseammengeführt

z = layers.Dense(10,
activation='LeakyReLU', #"tanh",#'relu',

)(merging_layer)

z = layers.Dropout(rate=0.5,


)(z)

z = layers.Dense(8,
activation='LeakyReLU', #"tanh",#'relu',

)(z)

z = layers.Dense(1,activation='sigmoid',
#kernel_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.01)
)(z)


merged_model = tf.keras.Model(inputs=[cnn.input,fnn.input],outputs=z)

In [ ]:
merged_model.summary()

In [ ]:
from keras.utils import plot_model


plot_model(merged_model, "my_first_model.png")

In [21]:
from keras import backend as K


merged_model.compile(
    loss="binary_crossentropy",
    metrics=#[ tf.keras.metrics.Precision(thresholds=[0.253]),tf.keras.metrics.Recall(thresholds=[0.253])],
	[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001) #lr
)

In [22]:
from functions import upload_tensorboard

from functions import upload_tensorboard

history = merged_model.fit(
    x=[X_train_img,X_train_metadata],
    y=y_train,
    batch_size=32,
    validation_data=(
        [X_val_img,X_val_metadata],y_val
        ),
    class_weight={0:1,1:10},    
    epochs=15,
    callbacks=[upload_tensorboard('','')])    


Epoch 1/15
843/843 [==============================] - 2502s 3s/step - loss: 0.6400 - recall: 0.3362 - precision: 0.1300 - val_loss: 0.2432 - val_recall: 0.3772 - val_precision: 0.0822
Epoch 2/15
843/843 [==============================] - 2458s 3s/step - loss: 0.5535 - recall: 0.4840 - precision: 0.1599 - val_loss: 0.1760 - val_recall: 0.2982 - val_precision: 0.1141
Epoch 3/15
843/843 [==============================] - 2463s 3s/step - loss: 0.5350 - recall: 0.5000 - precision: 0.1717 - val_loss: 0.2478 - val_recall: 0.4386 - val_precision: 0.0916
Epoch 4/15
843/843 [==============================] - 2459s 3s/step - loss: 0.5037 - recall: 0.5340 - precision: 0.1862 - val_loss: 0.2538 - val_recall: 0.4649 - val_precision: 0.0923
Epoch 5/15
843/843 [==============================] - 2470s 3s/step - loss: 0.4925 - recall: 0.5660 - precision: 0.1929 - val_loss: 0.2378 - val_recall: 0.5175 - val_precision: 0.0877
Epoch 6/15
843/843 [==============================] - 2460s 3s/step - loss: 0.47

Dieser Code oben ist Abgeändert von https://colab.research.google.com/drive/1jkgO5Rea-qbU-VVbbyoz_7jIPIoQXYjh?usp=sharing


In [10]:
merged_model.load_weights(r"trained_models\b3_upweighted10_weights") #hhier werden die trainierten Gewichte geladen

In [38]:
import os
npzs_path_to_predict =r"C:\Users\nilss\Desktop\benign_npzs/" #Der Pfad, welcher die dieDateien enthält die vorhergesagt werden sollen.
entries= os.listdir(npzs_path_to_predict)


(X_pred_img,X_pred_metadata), y_pred = get_X_y(entries,path=npzs_path_to_predict)    





In [12]:
merged_model.predict(  x=[X_pred_img,X_pred_metadata],
    batch_size=1 #Die Vorhersage.
    )

100/100 [==============================] - 7s 53ms/step


array([[2.8078834e-04],
       [1.8833237e-02],
       [1.2082694e-03],
       [1.6263379e-02],
       [4.1871507e-02],
       [8.8598847e-04],
       [1.7124253e-04],
       [5.0996870e-02],
       [3.4569690e-01],
       [9.8941580e-04],
       [6.2293666e-03],
       [2.9985708e-01],
       [2.5326600e-02],
       [5.8846754e-01],
       [3.9765739e-01],
       [4.0967536e-01],
       [1.9991906e-02],
       [3.5709262e-01],
       [6.2467833e-04],
       [7.8709645e-04],
       [2.4409866e-01],
       [8.0781607e-03],
       [4.0438924e-02],
       [2.9242354e-05],
       [1.5024922e-03],
       [1.0325306e-03],
       [2.9855827e-03],
       [3.6241983e-03],
       [5.2650428e-01],
       [6.4198560e-01],
       [1.6400197e-01],
       [5.0332387e-06],
       [1.8438365e-01],
       [4.5054424e-01],
       [3.3325371e-01],
       [2.2622234e-01],
       [2.3163746e-03],
       [9.2930272e-02],
       [5.7758544e-02],
       [1.6780731e-03],
       [2.5774583e-01],
       [3.515991